# "SQL"
> "Example database to learn and review SQL"

- author: Christopher Thiemann
- toc: true
- branch: master
- badges: true
- comments: true
- categories: [SQL, ]
- hide: false
- search_exclude: true

In [ ]:
#hide
import warnings


import numpy as np
import scipy as sp
import sklearn
import statsmodels.api as sm
from statsmodels.formula.api import ols


import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

# load specific python package
import sqlite3

In [ ]:
!curl https://cdn.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip -o database.zip
!unzip database.zip
!ls

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  298k  100  298k    0     0  1394k      0 --:--:-- --:--:-- --:--:-- 1394k
Archive:  database.zip
replace chinook.db? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: chinook.db              
chinook.db  database.zip  data.db  sample_data	test.zip


In [ ]:
con = sqlite3.connect("chinook.db")
cursor = con.cursor()
with con:
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table_name in cursor.fetchall():
        
        print(table_name[0])
        print(pd.read_sql_query(f"select * from {table_name[0]}", con).head(), "\n\n\n\n\n\n")

albums
   AlbumId                                  Title  ArtistId
0        1  For Those About To Rock We Salute You         1
1        2                      Balls to the Wall         2
2        3                      Restless and Wild         2
3        4                      Let There Be Rock         1
4        5                               Big Ones         3 






sqlite_sequence
          name   seq
0       genres    25
1  media_types     5
2      artists   275
3       albums   347
4       tracks  3503 






artists
   ArtistId               Name
0         1              AC/DC
1         2             Accept
2         3          Aerosmith
3         4  Alanis Morissette
4         5    Alice In Chains 






customers
   CustomerId  FirstName     LastName  \
0           1       Luís    Gonçalves   
1           2     Leonie       Köhler   
2           3   François     Tremblay   
3           4      Bjørn       Hansen   
4           5  František  Wichterlová   

                   

In [ ]:
pd.read_sql_query("select * from artists", con, index_col='ArtistId')

,Name
ArtistId,
1,AC/DC
2,Accept
3,Aerosmith
4,Alanis Morissette
5,Alice In Chains
...,...
271,"Mela Tenenbaum, Pro Musica Prague & Richard Kapp"
272,Emerson String Quartet
273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."


In [ ]:
pd.read_sql_query("select COUNT(AlbumId) as a from artists left join albums on artists.ArtistId = albums.ArtistId group by ArtistId", con)

DatabaseError: ignored

![](https://cdn.sqlitetutorial.net/wp-content/uploads/2018/03/sqlite-sample-database-diagram-color.pdf)

## Helper Functions

## Plot for the Blog Post

## Sources

- Hello This is a markdown page {% cite signaltrain %}
- http://openbook.rheinwerk-verlag.de/python/33_003.html#u33.3.1
- https://www.sqlitetutorial.net/sqlite-left-join/
- https://realpython.com/python-sqlite-sqlalchemy/

## References

{% bibliography --cited %}